In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import os
from config import project_dir,data_dir_raw
import pandas as pd
import dask_cudf



Transform to parquet

In [2]:
files= os.listdir(os.path.join(project_dir, data_dir_raw,'recsys23','train'))

In [3]:
for file in files:
    csv = cudf.read_csv(os.path.join(project_dir, data_dir_raw,'recsys23','train_csv',file),sep='\t')
    csv.to_parquet(os.path.join(project_dir, data_dir_raw,'recsys23','train',file.replace('.csv','.parquet')))

FileNotFoundError: /net/home/joao.miguel/Desktop/recsys23/./data/raw/recsys23/train_csv/000000000023.parquet could not be resolved to any files

In [9]:
files= os.listdir(os.path.join(project_dir, data_dir_raw,'recsys23','test_csv'))
for file in files:
    csv = cudf.read_csv(os.path.join(project_dir, data_dir_raw,'recsys23','test_csv',file),sep='\t')
    csv.to_parquet(os.path.join(project_dir, data_dir_raw,'recsys23','test',file.replace('.csv','.parquet')))

*** Start analysis of 01_first_eda.ipynb ***

In [14]:
# use cudf to read all parquet files in train
train = cudf.read_parquet(os.path.join(project_dir, data_dir_raw,'recsys23','train','*.parquet'))

In [16]:
# rename columns
train.rename(columns={'f_0':'row_id',
                      "f_1":"date"
                      },inplace=True)
dic = {}
for i in range(2, 33):
    dic[f"f_{i}"] = f"cat_{i-2}"
train.rename(columns=dic,inplace=True)
dic = {}
for i in range(33, 42):
    dic[f"f_{i}"] = f"bin_{i-33}"
train.rename(columns=dic,inplace=True)
dic = {}
for i in range(42, 80):
    dic[f"f_{i}"] = f"num_{i-42}"
train.rename(columns=dic,inplace=True)

In [17]:
train

,row_id,date,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,...,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,is_clicked,is_installed
0,2541162,51,6581,22294,5491,25604,14597,27941,18800,31372,...,3.997850,1.713364,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541260,49,20095,563,22861,15908,22651,27941,21218,869,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541318,51,20095,563,22861,25604,22651,27941,21218,31372,...,0.000000,0.000000,0.038564,1.156922,0.269948,0.0,0.0,0.0,0,0
3,2541770,64,13658,22294,7003,25604,29975,27941,21218,21533,...,1.713364,1.142243,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
4,2542002,62,20095,563,31686,15908,590,27941,18800,23218,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485847,1988041,62,12577,22294,2059,25604,18309,27941,21218,23218,...,0.000000,0.000000,0.000000,0.231384,0.000000,0.0,0.0,0.0,0,0
3485848,1988471,45,27426,22294,11338,21545,25991,27941,19203,6675,...,1.142243,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
3485849,1988651,56,26325,22294,9018,25604,943,27941,21218,869,...,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
3485850,1988792,48,31824,22294,6036,19475,9209,27941,19203,23218,...,2.284486,1.713364,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0


In [18]:
# remove columns with only one value
for col in train.columns:
    if len(train[col].unique()) == 1:
        train.drop(columns=[col],inplace=True)
        print(col)

cat_5


In [19]:
train.describe()

,row_id,date,cat_0,cat_1,cat_2,cat_3,cat_4,cat_6,cat_7,cat_8,...,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,is_clicked,is_installed
count,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,...,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06,3.485852e+06
mean,1.742926e+06,5.484742e+01,1.838541e+04,1.340448e+04,1.873755e+04,2.242766e+04,1.629934e+04,1.999042e+04,1.500072e+04,2.068723e+04,...,8.532780e-01,2.470630e-01,6.499100e-02,1.001575e+00,2.246020e-01,6.870200e-02,1.067790e+00,3.565410e-01,2.198350e-01,1.740180e-01
std,1.006279e+06,6.334539e+00,8.351711e+03,9.852940e+03,9.446633e+03,3.363930e+03,1.029889e+04,1.055990e+03,9.608741e+03,1.702075e+03,...,1.286822e+00,5.558180e-01,5.483600e-02,3.566770e-01,9.467500e-02,1.626738e+00,6.873551e+00,3.791883e+00,4.141350e-01,3.791250e-01
min,1.000000e+00,4.500000e+01,7.880000e+02,5.630000e+02,6.100000e+01,1.590800e+04,5.000000e+00,1.880000e+04,8.690000e+02,1.934300e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.714638e+05,4.900000e+01,1.239400e+04,5.630000e+02,1.129900e+04,2.154500e+04,6.707000e+03,1.920300e+04,6.675000e+03,1.934300e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,1.156922e+00,2.699480e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.742926e+06,5.500000e+01,2.009500e+04,2.229400e+04,1.897100e+04,2.154500e+04,1.862100e+04,1.920300e+04,1.465900e+04,1.934300e+04,...,5.711210e-01,0.000000e+00,1.156920e-01,1.156922e+00,2.699480e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.614389e+06,6.000000e+01,2.361100e+04,2.229400e+04,2.612400e+04,2.560400e+04,2.385500e+04,2.121800e+04,2.321800e+04,2.297000e+04,...,1.142243e+00,5.711210e-01,1.156920e-01,1.156922e+00,2.699480e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.485852e+06,6.600000e+01,3.268300e+04,2.229400e+04,3.274500e+04,2.930400e+04,3.276500e+04,2.162100e+04,3.137200e+04,2.297000e+04,...,5.711215e+00,3.997850e+00,1.156920e-01,1.156922e+00,2.699480e-01,1.121537e+02,4.859995e+02,2.243075e+02,1.000000e+00,1.000000e+00


In [32]:
train

,row_id,date,cat_0,cat_1,cat_2,cat_3,cat_4,cat_6,cat_7,cat_8,...,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,is_clicked,is_installed
0,2541162,51,6581,22294,5491,25604,14597,18800,31372,22970,...,3.997850,1.713364,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541260,49,20095,563,22861,15908,22651,21218,869,19343,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541318,51,20095,563,22861,25604,22651,21218,31372,22970,...,0.000000,0.000000,0.038564,1.156922,0.269948,0.0,0.0,0.0,0,0
3,2541770,64,13658,22294,7003,25604,29975,21218,21533,19343,...,1.713364,1.142243,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
4,2542002,62,20095,563,31686,15908,590,18800,23218,22970,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485847,1988041,62,12577,22294,2059,25604,18309,21218,23218,19343,...,0.000000,0.000000,0.000000,0.231384,0.000000,0.0,0.0,0.0,0,0
3485848,1988471,45,27426,22294,11338,21545,25991,19203,6675,22970,...,1.142243,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
3485849,1988651,56,26325,22294,9018,25604,943,21218,869,19343,...,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
3485850,1988792,48,31824,22294,6036,19475,9209,19203,23218,21574,...,2.284486,1.713364,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0


In [33]:
train.date.nunique()



22

In [11]:
# use nvt
train = nvt.Dataset(train)

45